# 13C-MFA Constrained Flux Balance Analysis (08302023 data + StrainDesign) 
The purpose of the notebook is to run flux balance analysis to find the set of genome scale fluxes that maximizes biomass production.<br><br>
The flux balance analysis results are then compared to reaction rates determined by 13C-metabolic flux analysis (13C-MFA). <br><br>
The results of this notebook will be compared to transcript constained genome scale model flux results via E-Flux2. <br><br>
This notebook looks at glucose, glycerol, acetate, and oleic acid as sole carbon sources. <br><br>
In all cases, parsimonious flux balance analysis (pFBA) was used to prevent degenerate solutions.


### Load imports

In [1]:
import cobra
import straindesign as sd
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)
from add_flux_column_to_13c_flux_df import add_flux_column_to_13c_flux_df
from add_fva_columns_to_13c_flux_df import add_fva_columns_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model('../genome_scale_models/iYLI647_corr_2.json')

### Rename some reactions to remove parentheses
This is because parentheses in reaction ids cause problems with StrainDesign

In [3]:
model.reactions.get_by_id('EX_glc(e)').id = 'EX_glc_e'
model.reactions.get_by_id('EX_glyc(e)').id = 'EX_glyc_e'
model.reactions.get_by_id('EX_ocdcea(e)').id = 'EX_ocdcea_e'
model.reactions.get_by_id('EX_h2o(e)').id = 'EX_h2o_e'
model.reactions.get_by_id('EX_h(e)').id = 'EX_h_e'
model.reactions.get_by_id('EX_nh4(e)').id = 'EX_nh4_e'
model.reactions.get_by_id('EX_o2(e)').id = 'EX_o2_e'
model.reactions.get_by_id('EX_pi(e)').id = 'EX_pi_e'
model.reactions.get_by_id('EX_so4(e)').id = 'EX_so4_e'

# print an example reaction
model.reactions.get_by_id('EX_glc_e')

Reaction identifier,EX_glc_e
Name,D Glucose exchange
Memory address,0x160065d80
Stoichiometry,glc_D[e] <=> D_Glucose <=>
GPR,YALI0D01111g or YALI0D18876g or YALI0D00132g or YALI0B01342g or YALI0E23287g or YALI0B00396g or...
Lower bound,-10.0
Upper bound,1000.0


### Load 13C-MFA data

In [6]:
# load glucose 13C MFA data from ../data/13c_mfa/INCA_model_08302023_GR.xlsx
central_rxn_df = pd.read_excel('../data/13c_mfa/INCA_model_08302023_GR.xlsx', sheet_name='GSM Format')

print(f'There are {len(central_rxn_df)} reactions in the 13C MFA that are mapped to the GSM')

central_rxn_df.head(10)

There are 73 reactions in the 13C MFA that are mapped to the GSM


,Unnamed: 0,ID,Equation,reaction_ids,pathway,compartment,glucose_flux,glucose_std_err,glucose_LB,glucose_UB,...,glycerol_flux,glycerol_std_err,glycerol_LB,glycerol_UB,glycerol_ΔB,oleic_acid_flux,oleic_acid_std_err,oleic_acid_LB,oleic_Acid_UB,oleic_acid_ΔB
0,uptake,uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,substrate_uptake,cytosol,100.0000,23879.0,100.0000,100.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,uptake,R3 glyc3p,GLYC + ATP -> Glyc3P,reverse_GLYCt,emp,cytosol,NaN,NaN,NaN,NaN,...,100.0000,4.911800e-10,100.0000,100.0000,0.0000,NaN,NaN,NaN,NaN,NaN
2,uptake,R3 dhap net,Glyc3P <-> DHAP + UQH2,NaN,emp,cytosol,NaN,NaN,NaN,NaN,...,100.0000,4.911800e-10,100.0000,100.0000,0.0000,NaN,NaN,NaN,NaN,NaN
3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 7*NADH + 7*FADH2,OCDCEAt,substrate_uptake,cytosol,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0000,2.201100e-11,0.0000,0.0000,0.0000
4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,emp,cytosol,17.8093,4252.8,13.4786,21.3566,...,-12.2900,5.856500e+00,-23.5032,-2.2969,21.2063,-151.6655,1.094870e+01,-165.8804,-1.3939,164.4865
5,glycolysis/gluconeogensis,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,emp,cytosol,68.0875,16259.0,66.6504,69.2727,...,-6.6656,1.910000e+00,-10.3212,-3.4075,6.9137,-62.8209,3.661100e+00,-68.0848,-14.6820,53.4028
6,glycolysis/gluconeogensis,R6 net,FBP <-> DHAP + GAP,FBA,emp,cytosol,68.0875,16259.0,66.6504,69.2727,...,-6.6656,1.910000e+00,-10.3212,-3.4075,6.9137,-62.8209,3.661100e+00,-68.0848,-14.6820,53.4028
7,glycolysis/gluconeogensis,R7 net,DHAP <-> GAP,TPI,emp,cytosol,68.0875,16259.0,66.6504,69.2727,...,93.3344,1.910000e+00,89.6788,96.5929,6.9141,-62.8209,3.661100e+00,-68.0848,-14.6820,53.4028
8,glycolysis/gluconeogensis,R8 net,GAP <-> G3P + ATP + NADH,GAPD,emp,cytosol,158.5742,37867.0,157.1426,159.7746,...,87.9222,1.821200e+00,84.4387,91.0275,6.5888,-88.6622,3.716800e+00,-95.2910,-44.5043,50.7867
9,glycolysis/gluconeogensis,R9 net,G3P <-> PEP,ENO,emp,cytosol,150.5627,35954.0,149.8699,151.3211,...,82.4971,1.755000e+00,79.0590,85.5244,6.4654,-118.0760,3.829800e+00,NaN,-78.6386,NaN


# Glucose

### Define functions to create a constraint string

In [73]:
def make_rxn_constraint_string(reaction_ids, lower_bound, upper_bound):
    or_split = [x.strip(' ') for x in reaction_ids.split(' or ')]
    
    if len(or_split) > 1:
        # Handle 'or' case
        reactions = []
        for index, reaction in enumerate(or_split):
            is_reverse = reaction.startswith('reverse_')
            reaction = reaction.replace('reverse_', '')
            if is_reverse:
                reactions.append(f' - {reaction}')
            elif index != 0:
                reactions.append(f' + {reaction}')
            else:
                reactions.append(reaction)
        reactions_str = ''.join(reactions)
        constraint_string = f'{reactions_str} >= {lower_bound}, {reactions_str} <= {upper_bound}'


    else:
        and_split = [x.strip(' ') for x in or_split[0].split(' and ')]
        if len(and_split) > 1:
            # Handle 'and' case
            constraints = []
            for reaction in and_split:
                is_reverse = reaction.startswith('reverse_')
                reaction = reaction.replace('reverse_', '')
                if is_reverse:
                    constraints.append(f'{reaction} >= {-1 * upper_bound}, {reaction} <= {-1 * lower_bound}')
                else:
                    constraints.append(f'{reaction} >= {lower_bound}, {reaction} <= {upper_bound}')
            constraint_string = ', '.join(constraints)
        else:
            # Handle single reaction case
            reaction = and_split[0]
            is_reverse = reaction.startswith('reverse_')
            reaction = reaction.replace('reverse_', '')
            if is_reverse:
                constraint_string = f'{reaction} >= {-1 * upper_bound}, {reaction} <= {-1 * lower_bound}'
            else:
                constraint_string = f'{reaction} >= {lower_bound}, {reaction} <= {upper_bound}'
    
    return constraint_string

# Test the function with examples
print(make_rxn_constraint_string('PGI', 0, 100))
print(make_rxn_constraint_string('PFK or reverse_FBP', 66.6504, 69.2727))
print(make_rxn_constraint_string('AKGDam and AKGDbm', 0, 100))

PGI >= 0, PGI <= 100
PFK - FBP >= 66.6504, PFK - FBP <= 69.2727
AKGDam >= 0, AKGDam <= 100, AKGDbm >= 0, AKGDbm <= 100


In [78]:
# def build_constraint_string():

lower_bound_row = 'glucose_LB'
upper_bound_row = 'glucose_UB'

constraint_strings = []

# loop over rows in the central flux dataframe
for _, row in central_rxn_df.iterrows():  
    

    # get the GSM reaction mapping for this reaction
    reaction_ids = row['reaction_ids']

    # get the lower bound for this reaction
    lower_bound = row[lower_bound_row]
    upper_bound = row[upper_bound_row]

    # check that are reaction ids
    if pd.isna(reaction_ids):
        # print(f'No reaction ids for {row["Equation"]}')
        print()

    # handle case where there are reaction ids
    else:             
        # Add the flux value for each row to the column values list
        print(reaction_ids)
        
        # check if lower bound and upper bound are not nan
        if pd.isna(lower_bound) or pd.isna(upper_bound):
            print('Either the lower bound or upper bound is nan')
        else: 
            print(f'13C-MFA bounds: {lower_bound} - {upper_bound}')

            # make the constraint string
            constraint_string = make_rxn_constraint_string(reaction_ids, lower_bound, upper_bound)

            # add the constraint string to the list
            constraint_strings.append(constraint_string)
            print(constraint_string)


        print()

full_constraint_string = ', '.join(constraint_strings)
full_constraint_string 

reverse_EX_glc_e
13C-MFA bounds: 100.0 - 100.0
EX_glc_e >= -100.0, EX_glc_e <= -100.0

reverse_GLYCt
Either the lower bound or upper bound is nan


OCDCEAt
Either the lower bound or upper bound is nan

PGI
13C-MFA bounds: 13.4786 - 21.3566
PGI >= 13.4786, PGI <= 21.3566

PFK or reverse_FBP
13C-MFA bounds: 66.6504 - 69.2727
PFK - FBP >= 66.6504, PFK - FBP <= 69.2727

FBA
13C-MFA bounds: 66.6504 - 69.2727
FBA >= 66.6504, FBA <= 69.2727

TPI
13C-MFA bounds: 66.6504 - 69.2727
TPI >= 66.6504, TPI <= 69.2727

GAPD
13C-MFA bounds: 157.1426 - 159.7746
GAPD >= 157.1426, GAPD <= 159.7746

ENO
13C-MFA bounds: 149.8699 - 151.3211
ENO >= 149.8699, ENO <= 151.3211

PYK
13C-MFA bounds: 135.5554 - 137.8096
PYK >= 135.5554, PYK <= 137.8096

PPCK
13C-MFA bounds: 0.0 - 1.058
PPCK >= 0.0, PPCK <= 1.058

G6PDH2
13C-MFA bounds: 78.6431 - 86.5215
G6PDH2 >= 78.6431, G6PDH2 <= 86.5215

GND
13C-MFA bounds: 78.6431 - 86.5215
GND >= 78.6431, GND <= 86.5215

reverse_RPI
13C-MFA bounds: 30.7273 - 33.3496
RPI >= -33

'EX_glc_e >= -100.0, EX_glc_e <= -100.0, PGI >= 13.4786, PGI <= 21.3566, PFK - FBP >= 66.6504, PFK - FBP <= 69.2727, FBA >= 66.6504, FBA <= 69.2727, TPI >= 66.6504, TPI <= 69.2727, GAPD >= 157.1426, GAPD <= 159.7746, ENO >= 149.8699, ENO <= 151.3211, PYK >= 135.5554, PYK <= 137.8096, PPCK >= 0.0, PPCK <= 1.058, G6PDH2 >= 78.6431, G6PDH2 <= 86.5215, GND >= 78.6431, GND <= 86.5215, RPI >= -33.3496, RPI <= -30.7273, RPE >= 47.9137, RPE <= 53.1734, TKT1 + TKT2 >= 47.9137, TKT1 + TKT2 <= 53.1734, TKT2 >= 20.4264, TKT2 <= 23.0651, TKT1 >= 27.4853, TKT1 <= 30.1098, TALA >= 27.4853, TALA <= 30.1098, TALA >= 27.4853, TALA <= 30.1098, PDHm >= 75.1116, PDHm <= 83.9326, CSm >= 45.9786, CSm <= 50.4378, ACONTm >= 23.2589, ACONTm <= 26.3246, ICDHxm >= 20.2194, ICDHxm <= 26.3246, ICDHym >= 0.0, ICDHym <= 3.4234, AKGDam >= 0.0, AKGDam <= 2.9799, AKGDbm >= 0.0, AKGDbm <= 2.9799, SUCD2_u6m >= 22.0621, SUCD2_u6m <= 27.2136, SUCD1m >= 22.0621, SUCD1m <= 27.2136, FUMm >= 28.4191, FUMm <= 33.3866, MDHm >= 45

### Calculate glucose GSM pFBA solution

In [79]:
# update the media to minimal medium with glucose as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 100
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
# constraint_string = 'EX_glc_e = -100.000'
# constraint_string += ', PGI >= 13.4786'
# constraint_string += ', PGI <= 21.3566'
# constraint_string += ', PGI <= 21.3566'
# constraint_string += ', PGI <= 21.3566'


# print(constraint_string)
# glucose_fba_solution = sd.fba(model, constraints='EX_glc_e = -100.000', obj='biomass_C', obj_sense='maximize', pfba=1)
print(full_constraint_string)
glucose_fba_solution = sd.fba(model, constraints=full_constraint_string, obj='biomass_C', obj_sense='maximize', pfba=1)

print()

max_glucose_biomass_flux = glucose_fba_solution['biomass_C']
PGI_flux = glucose_fba_solution['PGI']
print(f'Maximum biomass flux: {max_glucose_biomass_flux}.')
print(f'PGI flux: {PGI_flux}.')
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in glucose_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in glucose_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': flux,
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
glucose_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
glucose_gsm_fba_df = glucose_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
glucose_gsm_fba_df = glucose_gsm_fba_df.drop(columns=['absolute_flux'])

glucose_gsm_fba_df.head()

100 EX_glc_e
10000 EX_h2o_e
10000 EX_h_e
10000 EX_nh4_e
10000 EX_o2_e
10000 EX_pi_e
10000 EX_so4_e
EX_glc_e >= -100.0, EX_glc_e <= -100.0, PGI >= 13.4786, PGI <= 21.3566, PFK - FBP >= 66.6504, PFK - FBP <= 69.2727, FBA >= 66.6504, FBA <= 69.2727, TPI >= 66.6504, TPI <= 69.2727, GAPD >= 157.1426, GAPD <= 159.7746, ENO >= 149.8699, ENO <= 151.3211, PYK >= 135.5554, PYK <= 137.8096, PPCK >= 0.0, PPCK <= 1.058, G6PDH2 >= 78.6431, G6PDH2 <= 86.5215, GND >= 78.6431, GND <= 86.5215, RPI >= -33.3496, RPI <= -30.7273, RPE >= 47.9137, RPE <= 53.1734, TKT1 + TKT2 >= 47.9137, TKT1 + TKT2 <= 53.1734, TKT2 >= 20.4264, TKT2 <= 23.0651, TKT1 >= 27.4853, TKT1 <= 30.1098, TALA >= 27.4853, TALA <= 30.1098, TALA >= 27.4853, TALA <= 30.1098, PDHm >= 75.1116, PDHm <= 83.9326, CSm >= 45.9786, CSm <= 50.4378, ACONTm >= 23.2589, ACONTm <= 26.3246, ICDHxm >= 20.2194, ICDHxm <= 26.3246, ICDHym >= 0.0, ICDHym <= 3.4234, AKGDam >= 0.0, AKGDam <= 2.9799, AKGDbm >= 0.0, AKGDbm <= 2.9799, SUCD2_u6m >= 22.0621, SUCD2_

,reaction_id,reaction_name,full_reaction,flux
0,13BGH,Endo 1 3 beta glucan glucohydrase,13BDglcn[c] + h2o[c] --> glc_D[c],0.0
896,NTP4,nucleoside triphosphatase dGTP,dgtp[c] + h2o[c] --> dgdp[c] + h[c] + pi[c],0.0
904,MCITDm,2 methylcitrate dehydratase mitochondrial,hcit[m] <=> b124tc[m] + h2o[m],0.0
903,OAAt2m,oxaloacetate transport mitochondrial,h[c] + oaa[c] <=> h[m] + oaa[m],0.0
902,HICITDm,homoisocitrate dehydrogenase,hicit[m] + nad[m] <=> h[m] + nadh[m] + oxag[m],0.0


### Calculate glucose GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
glucose_fva_solution = sd.fva(
  model, 
  constraints=f'EX_glc_e = -100.000, biomass_C >= {biomass_fraction * max_glucose_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glucose_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glucose_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glucose_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glucose_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glucose_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glucose_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glucose_gsm_fba_df.to_csv('../results/gsm_fluxes/glucose_gsm_13C_fba.csv', index=False)

# display updated dataframe
glucose_gsm_fba_df


### Add glucose pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, 'glucose_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, f'glucose_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Glycerol

### Calculate glycerol GSM pFBA solution

In [ ]:
# update the media to minimal medium with glycerol as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 0
medium['EX_glyc_e'] = 100
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
glycerol_fba_solution = sd.fba(model, constraints='EX_glyc_e = -100.000', obj='biomass_C', obj_sense='maximize', pfba=1)

max_glycerol_biomass_flux = glycerol_fba_solution['biomass_C']
print(f'Maximum biomass flux: {max_glycerol_biomass_flux}.')
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in glycerol_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in glycerol_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': flux,
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
glycerol_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
glycerol_gsm_fba_df = glycerol_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
glycerol_gsm_fba_df = glycerol_gsm_fba_df.drop(columns=['absolute_flux'])

glycerol_gsm_fba_df.head()

### Calculate glycerol GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
glycerol_fva_solution = sd.fva(
  model, 
  constraints=f'EX_glyc_e = -100.000, biomass_C >= {biomass_fraction * max_glycerol_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glycerol_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glycerol_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glycerol_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glycerol_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glycerol_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glycerol_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glycerol_gsm_fba_df.to_csv('../results/gsm_fluxes/glycerol_gsm_13C_fba.csv', index=False)

# display updated dataframe
glycerol_gsm_fba_df

### Add glycerol pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, 'glycerol_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, f'glycerol_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Oleic Acid

### Calculate oleic acid GSM pFBA solution

In [ ]:
# update the media to minimal medium with oleic_acid as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 0
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 10 # this prevents overflow
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
oleic_acid_fba_solution = sd.fba(model, constraints='EX_ocdcea_e = -10.000', obj='biomass_C', obj_sense='maximize', pfba=1)

max_oleic_acid_biomass_flux = oleic_acid_fba_solution['biomass_C']
print(f'Maximum biomass flux: {10 * max_oleic_acid_biomass_flux}.') # restore 100 input flux
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in oleic_acid_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in oleic_acid_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': 10 * flux, # restore 100 input flux
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
oleic_acid_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.drop(columns=['absolute_flux'])

oleic_acid_gsm_fba_df.head()

### Calculate oleic acid GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
oleic_acid_fva_solution = sd.fva(
  model, 
  constraints=f'EX_ocdcea_e = -10.000, biomass_C >= {biomass_fraction * max_oleic_acid_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in oleic_acid_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in oleic_acid_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = 10 * oleic_acid_fva_solution.loc[reaction_id, 'maximum'] # restore 100 input flux
  lower_bound = 10 * oleic_acid_fva_solution.loc[reaction_id, 'minimum'] # restore 100 input flux

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
oleic_acid_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
oleic_acid_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
oleic_acid_gsm_fba_df.to_csv('../results/gsm_fluxes/oleic_acid_gsm_13C_fba.csv', index=False)

# display updated dataframe
oleic_acid_gsm_fba_df


### Add oleic acid pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, 'oleic_acid_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, f'oleic_acid_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Save Data

### Save central flux data with pFBA data added

In [ ]:
# save the dataframe to a csv file
central_rxn_df.to_csv('../results/central_fluxes/13C_pfba.csv', index=False, encoding='utf-8-sig')